In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['gender_submission.csv', 'test.csv', 'train.csv']


In [2]:
#Load training set
df_raw_train = pd.read_csv('../input/train.csv')
df_raw_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Load testing set
df_raw_test = pd.read_csv('../input/test.csv')
df_raw_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_raw_train.isnull().sum().sort_index()

Age            177
Cabin          687
Embarked         2
Fare             0
Name             0
Parch            0
PassengerId      0
Pclass           0
Sex              0
SibSp            0
Survived         0
Ticket           0
dtype: int64

In [5]:
train_data = df_raw_train.copy()
train_data["Age"].fillna(df_raw_train["Age"].median(skipna=True), inplace=True)
train_data["Embarked"].fillna(df_raw_train['Embarked'].value_counts().idxmax(), inplace=True)
train_data.drop('Cabin', axis=1, inplace=True)


In [6]:
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [7]:
data_cols = train_data[['Pclass','Sex','Age','Fare','SibSp', 'Parch','Embarked']]
X = pd.get_dummies(data_cols)
print(X.head())
y = train_data['Survived']
print(y.head())

   Pclass   Age     Fare     ...      Embarked_C  Embarked_Q  Embarked_S
0       3  22.0   7.2500     ...               0           0           1
1       1  38.0  71.2833     ...               1           0           0
2       3  26.0   7.9250     ...               0           0           1
3       1  35.0  53.1000     ...               0           0           1
4       3  35.0   8.0500     ...               0           0           1

[5 rows x 10 columns]
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [8]:
X.isnull().sum()

Pclass        0
Age           0
Fare          0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [9]:
#dividing the data in training and test data 

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
logreg = LogisticRegression() #logistic regression using python
my_model = logreg.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
from sklearn.metrics import accuracy_score
y_pred = logreg.predict(X_val) #predicting the values
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_val, y_pred)))

Accuracy of logistic regression classifier on test set: 0.80


In [11]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(my_model, random_state=1).fit(X_val, y_val)
eli5.show_weights(perm, feature_names = X_val.columns.tolist())

Weight,Feature
0.1587 ± 0.0456,Sex_female
0.0458 ± 0.0286,Pclass
0.0291 ± 0.0342,Age
0.0145 ± 0.0195,SibSp
0.0134 ± 0.0055,Fare
0.0123 ± 0.0084,Embarked_S
0.0123 ± 0.0205,Embarked_C
0.0022 ± 0.0182,Sex_male
0.0011 ± 0.0109,Embarked_Q
-0.0011 ± 0.0045,Parch
